In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os

In [2]:
df = pd.read_csv(r'C:\Users\Norah\OneDrive\Documents\Inputs_LandUse_E_All_Data_(Normalized).csv', low_memory=False)


In [3]:
df['Element Code'] = pd.to_numeric(df['Element Code'], errors='coerce')
dfelement = df[df['Element Code'] == 5110]

In [4]:
dfdrop = ['Element Code', 'Element', 'Area Code', 'Area Code (M49)', 'Item', 'Year Code', 'Unit', 'Flag', 'Note']
df1 = dfelement.drop(columns=dfdrop)
df1



,Area,Item Code,Year,Value
0,Afghanistan,6600,1961,65286.000
1,Afghanistan,6600,1962,65286.000
2,Afghanistan,6600,1963,65286.000
3,Afghanistan,6600,1964,65286.000
4,Afghanistan,6600,1965,65286.000
...,...,...,...,...
407807,Net Food Importing Developing Countries (NFIDCs),6714,2013,259510.905
407808,Net Food Importing Developing Countries (NFIDCs),6714,2014,258504.912
407809,Net Food Importing Developing Countries (NFIDCs),6714,2015,257498.919
407810,Net Food Importing Developing Countries (NFIDCs),6714,2016,257498.919


In [5]:
df1['Item Code'] = pd.to_numeric(df1['Item Code'], errors='coerce')
item_filters = {
    '6600': df1[df1['Item Code'] == 6600],
    '6610': df1[df1['Item Code'] == 6610],
    '6601': df1[df1['Item Code'] == 6601],
    '6655': df1[df1['Item Code'] == 6655],
    '6670': df1[df1['Item Code'] == 6670],
    '6620': df1[df1['Item Code'] == 6620],
    '6646': df1[df1['Item Code'] == 6646],
    '6611': df1[df1['Item Code'] == 6611],
}

In [14]:
with pd.ExcelWriter('FAOLand.xlsx') as writer:
    for code, data in item_filters.items():
        sheet_name = f'Item_{code}'
        data.to_excel(writer, sheet_name=sheet_name, index=False)


In [15]:
df = pd.ExcelFile('FAOLand.xlsx')
df.sheet_names

['Item_6600',
 'Item_6610',
 'Item_6601',
 'Item_6655',
 'Item_6670',
 'Item_6620',
 'Item_6646',
 'Item_6611']

In [110]:
df = pd.read_excel('FAOLand.xlsx', sheet_name='Item_6600')
df = df.drop(columns=['Item Code'])
country_map = {
    "Ethiopia PDR": "Ethiopia",
}
df['Area'] = df['Area'].replace(country_map)
#['6600', '6601', '6655', '6670', '6620', '6646'] need to be divided by 1000
df['Value'] = df['Value']/1000
df = df.pivot_table(index='Area', columns='Year', values='Value')
df = df.reset_index()
df

Year,Area,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.286000,65.287000
1,Africa,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3031.837500,3035.440916,3035.440820,3035.437795,3035.434578,3035.436248,3035.437480,3035.436948,3035.461502,3037.368165,3037.365951,3037.364601,3037.370061,3037.368794,3037.368222,3037.368839,3037.366421,3038.521429,3038.534976,3038.535276,3038.535676
2,Albania,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000,2.875000
3,Algeria,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174000,238.174100,238.174100,238.174100,238.174100,238.174000,238.174000,238.174000,238.174000,238.174000
4,American Samoa,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000
5,Americas,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.326400,4616.325400,4616.325400,4616.32

['Item_6600',
 'Item_6610',
 'Item_6601',
 'Item_6655',
 'Item_6670',
 'Item_6620',
 'Item_6646',
 'Item_6611']

decimal points
LandTotal = 5 = 6600

LandArea = 4 = 6601

LandGrazing = 5 = 6655

LandOther = 5 = 6670

LandAgri = 3 = 6610

LandCrop = 3 =6620

LandForest = 5 = 6646

LandIrArea = 3 = 6611

LandOther% = 3 = 6670 %

LandPasture% = 3 = 6655 %


In [111]:
country = pd.read_excel('FAO Country.xlsx')
dt = country.merge(df, on='Area', how='left')
dt = dt.drop(columns=['Area'])

year_columns = [col for col in dt.columns if str(col).isdigit()]
dt['Earliest'] = dt[year_columns].bfill(axis=1).iloc[:, 0]
dt['MostRecent'] = dt[year_columns].ffill(axis=1).iloc[:, -1]

#change decimal place according to the datadict

dt[year_columns + ['Earliest', 'MostRecent']] = dt[year_columns + ['Earliest', 'MostRecent']].round(5)
dt

,Country,FIPS_CODE,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,Earliest,MostRecent
0,Afghanistan,AFG,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28600,65.28700,65.28600,65.28700
1,Albania,ALB,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500,2.87500
2,Algeria,DZA,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17410,238.17410,238.17410,238.17410,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400,238.17400
3,Angola,AGO,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000,124.67000
4,Argentina,ARG,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000,278.04000
5,Armenia,ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400,2.97400
6,Australia,AUS,774.12200,774.12200,774.12200,774.12200,774.12200,774.12200,774.12200,774.12200,774.12200,

In [ ]:
#sql table; change table name each time you import 
conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport.db')
cursor = conn.cursor()
table_name = 'SeriesLandAgri1'
columns_sql = ['Country VARCHAR(255)', 'FIPS_CODE VARCHAR(255)']
columns_sql += [f'"{col}" DOUBLE(53)' for col in year_columns + ['Earliest', 'MostRecent']]
create_table_sql = f"CREATE TABLE '{table_name}' ({', '.join(columns_sql)})"
cursor.execute(f'DROP TABLE IF EXISTS "{table_name}"')
cursor.execute(create_table_sql)

dt.to_sql(table_name, conn, if_exists='append', index=False)

conn.commit()
conn.close()

what i have to do for 
USSR = Armenia, Azerbaijan, Belarus, Estonia, Georgia, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Moldova, Russia, Tajikistan, Turkmenistan, Ukraine, and Uzbekistan
Czechoslovakia = Slovakia and Czech Republic
Yugoslav SFR = Slovenia, Croatia, Bosnia and Herzegovina, North Macedonia, Serbia, and Montenegro
Serbia and Montenegro = Serbia and Montenegro
Sudan (former) = Sudan and South Sudan

select USSR, Sudan (former), Serbia and Montenegro, Czechoslovakia, Yugoslav SFR from the FAO data

USSR add up the values for 1992; have totals for 1961-1991; have the values for 15 states for 1991; extrapolate data for 15 states 1961-1991

Czechoslovakia add up the values for for 1993; have totals for 1961-1992; have the values for 2 states for 1992; extrapolate data for 2 states 1961-1992

Sudan (former) add up the values for 2012; have totals for 1961-2011; have the values for 2 states for 2011; extrapolate data for 2 states 1961-2011

Yugoslav SFR add up the values for 1992; have totals for 1961-1991; have the values for 6 states for 1991; extrapolate for 6 states 1961-1991

Serbia and Montenegro add up the values for 2006; have totals for 1992-2005; have the values for 2 states for 2005; extrapolate data for 2 states 1992-2005

['Item_6600',
 'Item_6610',
 'Item_6601',
 'Item_6655',
 'Item_6670',
 'Item_6620',
 'Item_6646',
 'Item_6611']

decimal points
LandTotal = 5 = 6600

LandArea = 4 = 6601

LandGrazing = 5 = 6655

LandOther = 5 = 6670

LandAgri = 3 = 6610

LandCrop = 3 =6620

LandForest = 5 = 6646

LandIrArea = 3 = 6611

LandOther% = 3 = 6670 %

LandPasture% = 3 = 6655 %


In [108]:
#['6600', '6601', '6655', '6670', '6620', '6646'] need to be divided by 1000
#6610 do not have to be divided by 1000
#6611 does not have data for these items
df = pd.read_excel('FAOLand.xlsx', sheet_name='Item_6600')
df['Value'] = df['Value']/1000

df = df.pivot_table(index='Area', columns='Year', values='Value')
df.index.name = 'Country'
year_columns = [col for col in df.columns if isinstance(col, int)]

ussr = 'USSR'
ussr_republics = [
    "Armenia", "Azerbaijan", "Belarus", "Estonia", "Georgia", "Kazakhstan",
    "Kyrgyzstan", "Latvia", "Lithuania", "Republic of Moldova", "Russian Federation", "Tajikistan",
    "Turkmenistan", "Ukraine", "Uzbekistan"
]
ussr_split_year = 1992
ussr_pre_split_years = [y for y in year_columns if 1961 <= y < ussr_split_year]
ussr_totals = df.loc[ussr, ussr_pre_split_years].tolist()
ussr_republic_vals = df.loc[ussr_republics, ussr_split_year].dropna()
ussr_total_1992 = ussr_republic_vals.sum()
ussr_shares = (ussr_republic_vals / ussr_total_1992).to_dict()
df_ussr = pd.DataFrame({
    country: [round(total * ussr_shares[country], 5) for total in ussr_totals]
    for country in ussr_shares
}, index=ussr_pre_split_years).T
df_ussr.index.name = 'Country'

cz = 'Czechoslovakia'
cz_republics = ['Czechia', 'Slovakia']
cz_split_year = 1993
cz_pre_split_years = [y for y in year_columns if 1961 <= y < cz_split_year]
cz_totals = df.loc[cz, cz_pre_split_years].tolist()
cz_republic_vals = df.loc[cz_republics, cz_split_year].dropna()
cz_total = cz_republic_vals.sum()
cz_shares = (cz_republic_vals / cz_total).to_dict()
df_cz = pd.DataFrame({
    country: [round(total * cz_shares[country], 5) for total in cz_totals]
    for country in cz_shares
}, index=cz_pre_split_years).T
df_cz.index.name = 'Country'

sudan = 'Sudan (former)'
sudan_republics = ['Sudan', 'South Sudan']
sudan_split_year = 2012
sudan_pre_split_years = [y for y in year_columns if 1961 <= y < sudan_split_year]
sudan_totals = df.loc[sudan, sudan_pre_split_years].tolist()
sudan_republic_vals = df.loc[sudan_republics, sudan_split_year].dropna()
sudan_total = sudan_republic_vals.sum()
sudan_shares = (sudan_republic_vals / sudan_total).to_dict()
df_sudan = pd.DataFrame({
    country: [round(total * sudan_shares[country], 5) for total in sudan_totals]
    for country in sudan_shares
}, index=sudan_pre_split_years).T
df_sudan.index.name = 'Country'

scg = 'Serbia and Montenegro'
scg_republics = ['Serbia', 'Montenegro']
scg_split_year = 2006
scg_pre_split_years = [y for y in year_columns if 1992 <= y < scg_split_year]
scg_totals = df.loc[scg, scg_pre_split_years].tolist()
scg_republic_vals = df.loc[scg_republics, scg_split_year].dropna()
scg_total = scg_republic_vals.sum()
scg_shares = (scg_republic_vals / scg_total).to_dict()
df_scg = pd.DataFrame({
    country: [round(total * scg_shares[country], 5) for total in scg_totals]
    for country in scg_shares
}, index=scg_pre_split_years).T
df_scg.index.name = 'Country'

yugo = 'Yugoslav SFR'
yugo_republics = ['Slovenia', 'Croatia', 'Bosnia and Herzegovina',
                  'North Macedonia', 'Serbia', 'Montenegro']
yugo_split_year = 1992
yugo_pre_split_years = [y for y in year_columns if 1961 <= y < yugo_split_year]
yugo_totals = df.loc[yugo, yugo_pre_split_years].tolist()

#for 1992 yugoslov to make sure serbia and montengro are in there 
yugo_republic_vals = df.loc[yugo_republics, yugo_split_year].copy()
yugo_republic_vals.loc['Serbia'] = df_scg.loc['Serbia', yugo_split_year]
yugo_republic_vals.loc['Montenegro'] = df_scg.loc['Montenegro', yugo_split_year]

yugo_total_1992 = yugo_republic_vals.sum()
yugo_shares = (yugo_republic_vals / yugo_total_1992).to_dict()

df_yugo = pd.DataFrame({
    country: [round(total * yugo_shares[country], 5) for total in yugo_totals]
    for country in yugo_shares
}, index=yugo_pre_split_years).T
df_yugo.index.name = 'Country'

with pd.ExcelWriter("E6600.xlsx") as writer:
    df_ussr.to_excel(writer, sheet_name="USSR")
    df_cz.to_excel(writer, sheet_name="Czechoslovakia")
    df_sudan.to_excel(writer, sheet_name="Sudan")
    df_yugo.to_excel(writer, sheet_name="Yugoslav_SFR")
    df_scg.to_excel(writer, sheet_name="Serbia_Montenegro")

LandTotal
LandArea
LandGrazing
LandOther
LandAgri
LandCrop
LandForest
after calculations

update earliest and most recent values


In [123]:
conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport.db')

dt = pd.read_sql_query("SELECT * FROM SeriesLandForest", conn)

year_columns = [col for col in dt.columns if str(col).isdigit()]

dt['Earliest'] = dt[year_columns].bfill(axis=1).iloc[:, 0]
dt['MostRecent'] = dt[year_columns].ffill(axis=1).iloc[:, -1]

dt.to_sql('SeriesLandForest', conn, if_exists='replace', index=False)

conn.close()

the % tables = do after calculations to not do more

LandOther%	Land use, other land (% of total area)	LandOther/LandTotal

LandPasture%	Land use, permanent pasture (% of total area)	LandGrazing/LandTotal

In [127]:
tables = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport.db')
cursor = tables.cursor()
landother = pd.read_sql_query("SELECT * FROM SeriesLandOther", tables)
landgrazing = pd.read_sql_query("SELECT * FROM SeriesLandGrazing", tables)
landtotal = pd.read_sql_query("SELECT * FROM SeriesLandTotal", tables)

#country
landothercountry = landother[['Country', 'FIPS_CODE']]
landgrazingcountry = landgrazing[['Country', 'FIPS_CODE']]
landtotalcountry = landtotal[['Country', 'FIPS_CODE']]

#convert to numeric
landother1 = landother.apply(pd.to_numeric, errors='coerce')
landgrazing1 = landgrazing.apply(pd.to_numeric, errors='coerce')
landtotal1 = landtotal.apply(pd.to_numeric, errors='coerce')

#landother%
landtotal1 = landtotal.iloc[:, 2:]
landother1 = landother.iloc[:, 2:]

landothernlandtotal = landother1.copy()
landothernlandtotal = landother1 / landtotal1 * 100
#drop 1961-1989
landothernlandtotal = landothernlandtotal.dropna(axis=1, how='all')

#landgrazing%
landgrazing1 = landgrazing.iloc[:, 2:]
landtotal1 = landtotal.iloc[:, 2:]

landgrazingnlandtotal = landgrazing1.copy()
landgrazingnlandtotal = landgrazing1 / landtotal1 *100
 
#adding countries back
landothernlandtotal = pd.concat([landothercountry, landothernlandtotal], axis=1)
landgrazingnlandtotal = pd.concat([landgrazingcountry, landgrazingnlandtotal], axis=1)

landgrazingnlandtotal = landgrazingnlandtotal.round(3)
landothernlandtotal = landothernlandtotal.round(3)


landothernlandtotal.to_sql('SeriesLandOther%', tables, if_exists='replace', index=False)
landgrazingnlandtotal.to_sql('SeriesLandGrazing%', tables, if_exists='replace', index=False)

tables.commit()
tables.close() 